In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pyodbc

# Import TSNE
from sklearn.manifold import TSNE

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_dincident,inc_time,inc_dreported,inc_submittedtime,inc_organisation,inc_locactual,inc_unit,inc_specialty,inc_loctype,inc_result,inc_severity,show_other_contacts,show_employee,show_witness,show_document,inc_reportedby,inc_notes from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC'"
df = pd.read_sql(query, sql_conn)
df.head()

In [ ]:
df['division'] = df['inc_unit'].str[:3]
df['care group'] = df['inc_unit'].str[3:6]
df = df.drop('inc_unit',axis=1)

In [ ]:
df.columns = [column.replace('inc_organisation','site') for column in df.columns]
df.columns = [column.replace('inc_locactual','ward/dept/unit') for column in df.columns]
df.columns = [column.replace('inc_unit','division & care group') for column in df.columns]
df.columns = [column.replace('inc_specialty','specialty') for column in df.columns]
df.columns = [column.replace('inc_loctype','location type') for column in df.columns]
df.columns = [column.replace('inc_result','result') for column in df.columns]
df.columns = [column.replace('inc_severity','severity') for column in df.columns]
df.columns = [column.replace('inc_reportedby','reported by') for column in df.columns]
df.columns = [column.replace('show_other_contacts','other patients involved?') for column in df.columns]
df.columns = [column.replace('show_employee','other employees involved?') for column in df.columns]
df.columns = [column.replace('show_witness','any witnesses?') for column in df.columns]
df.columns = [column.replace('show_document','any documents attached?') for column in df.columns]
df.head()

In [ ]:
df['inc_time'] = df['inc_time'].str.replace(':','')
df['inc_submittedtime'] = df['inc_submittedtime'].str.replace(':','')

In [ ]:
df['inc_year'] = df.inc_dincident.dt.year
df['inc_month'] = df.inc_dincident.dt.month
df['inc_day'] = df.inc_dincident.dt.day
df['inc_week'] = df.inc_dincident.dt.week
df['inc_weekday'] = df.inc_dincident.dt.weekday
df['inc_quarter'] = df.inc_dincident.dt.quarter
df['inc_hour'] = df['inc_time'].str[:2]
df['inc_minute'] = df['inc_time'].str[2:4]
df['inc_time_datetime'] = pd.to_datetime(df['inc_time'], format='%H%M')
df['inc_time_timedelta'] = pd.to_timedelta(df['inc_time_datetime']) + pd.Timedelta(days=25567)
df['inc_datetime'] = df['inc_dincident'] + df['inc_time_timedelta']
df['inc_date_delta'] = (df['inc_datetime'] - df['inc_datetime'].min())  / np.timedelta64(1,'D')
df['reported_year'] = df.inc_dreported.dt.year
df['reported_month'] = df.inc_dreported.dt.month
df['reported_day'] = df.inc_dreported.dt.day
df['reported_week'] = df.inc_dreported.dt.week
df['reported_weekday'] = df.inc_dreported.dt.weekday
df['reported_quarter'] = df.inc_dincident.dt.quarter
df['reported_hour'] = df['inc_submittedtime'].str[:2]
df['reported_minute'] = df['inc_submittedtime'].str[2:4]
df['reported_time_datetime'] = pd.to_datetime(df['inc_submittedtime'], format='%H%M')
df['reported_time_timedelta'] = pd.to_timedelta(df['reported_time_datetime']) + pd.Timedelta(days=25567)
df['reported_datetime'] = df['inc_dreported'] + df['reported_time_timedelta']
df['reported_date_delta'] = (df['reported_datetime'] - df['reported_datetime'].min())  / np.timedelta64(1,'D')
df['diff'] = df['reported_datetime'] - df['inc_datetime']
df['reported-incident'] = df['diff'] / np.timedelta64(1, 'D')
df = df.astype({'inc_hour':'int'})
df = df.astype({'inc_minute':'int'})
df = df.astype({'reported_hour':'int'})
df = df.astype({'reported_minute':'int'})
df = df.drop(['inc_dincident','inc_time','inc_dreported','inc_submittedtime','inc_time_datetime','inc_time_timedelta','inc_datetime','reported_time_datetime','reported_time_timedelta','reported_datetime','diff'],axis=1)
print(df.columns)
df = df.drop(['inc_year', 'inc_month', 'inc_day',
       'inc_week', 'inc_quarter', 'inc_minute',
       'reported_year', 'reported_month', 'reported_day',
       'reported_week', 'reported_weekday', 'reported_quarter',
       'reported_hour', 'reported_minute', 'reported_date_delta',
       'reported-incident'],axis=1)
df.head()

In [ ]:
result_classes = [str(i) for i in df['result'].value_counts().index]
result_mapping = {'NMISS':0,'NOHARM':1,'HARM':2}
df_replaced = df.replace({'result': result_mapping})

severity_classes = [str(i) for i in df['severity'].value_counts().index]
severity_mapping = {'NMISS':0,'NONE':1,'LOW':2,'MODRTE':3,'SEVERE':4,'':None}
df_replaced = df_replaced.replace({'severity': severity_mapping})

binary_mapping = {'N':0,'Y':1}
df_replaced = df_replaced.replace({'other patients involved?': binary_mapping})
df_replaced = df_replaced.replace({'other employees involved?': binary_mapping})
df_replaced = df_replaced.replace({'any witnesses?': binary_mapping})
df_replaced = df_replaced.replace({'any documents attached?': binary_mapping})
df_replaced.head()

In [ ]:
df_dropped = df_replaced.dropna()
df_dropped.head()

In [ ]:
df_num = df_dropped.select_dtypes(np.number)
print(df_num.columns)
df_num.head()

In [ ]:
# Create a TSNE instance: model
model = TSNE(learning_rate=200)

# Apply fit_transform to samples: tsne_features
tsne_features = model.fit_transform(df_num)

In [ ]:
# Select the 0th feature: xs
xs = tsne_features[:,0]

# Select the 1st feature: ys
ys = tsne_features[:,1]

In [ ]:
category_columns = list(df_replaced.select_dtypes('object'))
category_columns.remove('inc_notes')
for column in category_columns:
    classes = [str(i) for i in df_dropped[column].value_counts().index]
    mapping = dict((el,i) for i,el in enumerate(classes))
    df_mapped = df_dropped.replace({column: mapping})
    
    fig, ax = plt.subplots()
    
    # Scatter plot, coloring by column
    scatter = ax.scatter(xs,ys,c=df_mapped[column].values,label=classes)

    # produce a legend with a cross section of sizes from the scatter
    handles, labels = scatter.legend_elements(prop="colors")
    legend2 = ax.legend(handles, mapping.keys(), loc=(1.04,0))